# amazon Food Review


In [1]:
import pandas as pd

data=pd.read_csv("amazonFoodReview/Reviews.csv")
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
import sqlite3 
import pandas as pd 
import numpy as np 
import string 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem import PorterStemmer

In [4]:
# using the sqlite table to read data
con = sqlite3.connect("amazonFoodReview/database.sqlite")

In [5]:
#filtering only positive and negative reviews
# not taking into consideration those review with score =3


filtered_data='''select * from Reviews '''
pd.read_sql(filtered_data,con)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


# cleaning data

In [6]:
#filtering only positive and negative reviews
# not taking into consideration those review with score =3

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, con) 
#filtered_data='''select * from reviews where score!=3'''
#pd.read_sql(filtered_data,con)
print(filtered_data)

            Id   ProductId          UserId                      ProfileName  \
0            1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1            2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2            3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3            4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4            5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...        ...         ...             ...                              ...   
525809  568450  B001EO7N10  A28KG5XORO54AY                 Lettie D. Carter   
525810  568451  B003S1WTCU  A3I8AFVPEE8KI5                        R. Sawyer   
525811  568452  B004I613EE  A121AA1GQV751Z                    pksd "pk_007"   
525812  568453  B004I613EE   A3IBEVCTXKNOH          Kathy A. Welch "katwel"   
525813  568454  B001LR2CU2  A3LGQPJCZVL9UC                         srfell17   

        HelpfulnessNumerator  HelpfulnessDenominato

In [7]:
# give reviews with score>3 a positive rating and reviews with a score<3 a negative rating
def partition(x):
    if x<3:
        return "negative"
    return "positive"

In [8]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
#print(actualScore)
positiveNegative = actualScore.map(partition)  
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(13)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,positive,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,positive,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,positive,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,positive,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [9]:
display=pd.read_sql_query('''select * from Reviews where Score!=3 and UserId= 'AR5J8U746CURR' order by ProductId''',con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text


In [10]:
#sorting data according to productId in ascending order
Sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False,kind='quicksort', na_position='last',ignore_index=False)

In [11]:
# deduplication of entries
final=Sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'}, keep='first',inplace=False)
final.shape
print(final)

            Id   ProductId          UserId                  ProfileName  \
138706  150524  0006641040   ACITT7DI6IDDL              shari zychinski   
138688  150506  0006641040  A2IW4PEEKO2R0U                        Tracy   
138689  150507  0006641040  A1S4A3IQ2MU7V4        sally sue "sally sue"   
138690  150508  0006641040     AZGXZ2UUK6X  Catherine Hallberg "(Kate)"   
138691  150509  0006641040  A3CMRKGE0P909G                       Teresa   
...        ...         ...             ...                          ...   
178145  193174  B009RSR8HO   A4P6AN2L435PV                       romarc   
173675  188389  B009SF0TN6  A1L0GWGRK4BYPT                Bety Robinson   
204727  221795  B009SR4OQ2  A32A6X5KCP7ARG                      sicamar   
5259      5703  B009WSNWC4   AMP7K1O84DH1T                         ESTY   
302474  327601  B009WVB40S  A3ME78KVX31T21                         K'la   

        HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
138706               

In [12]:
# checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [13]:
# observation-: it was also seen that in two rows given the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows are typed wrongly so need to remove from calculation
display =pd.read_sql_query('''select * from Reviews where Score!=3 and Id =44737 or Id=64422 order by ProductId''',con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [14]:
# cleaning rows where HelpfulnessNumerator > HelpfulnessDenominator
final=final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
print(final.shape)

(364171, 10)


In [15]:
# how many positive and negative reviews are present in our dataset
final['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

In [16]:
#printing some random reviews
sent_0=final['Text'].values[0]
print(sent_0)
print('='*100)

sent_1000=final['Text'].values[1000]
print(sent_1000)
print('='*100)

sent_1500=final['Text'].values[1500]
print(sent_1500)
print('='*100)

sent_4900=final['Text'].values[4900]
print(sent_4900)
print('='*100)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.
Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a do

In [17]:
# Remove urls from text python: stackoverflow.com/a/40823105/4084039
import re

sent_0=re.sub(r'http\s+','', sent_0)
sent_1000=re.sub(r'http\s+','', sent_1000)
sent_1500=re.sub(r'http\s+','', sent_1500)
sent_4900=re.sub(r'http\s+','', sent_4900)

print(sent_0)
print('='*100)
print(sent_1000)
print('='*100)
print(sent_1500)
print('='*100)
print(sent_4900)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.
Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a do

In [18]:
# how to remove html tags from an element?
from bs4 import BeautifulSoup

soup=BeautifulSoup(sent_0,'lxml')
text=soup.get_text()
print(text)
print('='*50)

soup=BeautifulSoup(sent_1000,'lxml')
text=soup.get_text()
print(text)
print('='*50)

soup=BeautifulSoup(sent_1500,'lxml')
text=soup.get_text()
print(text)
print('='*50)

soup=BeautifulSoup(sent_4900,'lxml')
text=soup.get_text()
print(text)
print('='*50)


this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.
Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a do

In [19]:
# decontraction of phrase like 'won't' will converted to 'will not' and similar many more
def decontracted(phrase):
    #specific
    #phrase = re.sub(r"won\'t", "will not", phrase)
    phrase=re.sub(r"won\'t","will not",phrase)
    phrase=re.sub(r"can\'t","can not",phrase)
    
    #general
    phrase=re.sub(r"n\'t","not" ,phrase)
    phrase=re.sub(r"n\'re","are" ,phrase)
    phrase=re.sub(r"n\'s","is" ,phrase)
    phrase=re.sub(r"n\'d","would" ,phrase)
    phrase=re.sub(r"n\'ll","will" ,phrase)
    phrase=re.sub(r"n\'t","not" ,phrase)
    phrase=re.sub(r"n\'ve","have" ,phrase)
    phrase=re.sub(r"n\'m","am" ,phrase)
    
    return phrase

In [20]:
sent_1500=decontracted(sent_1500)
print(sent_1500)
print('='*50)

sent_4900=decontracted(sent_4900)
print(sent_4900)
print('='*50)

soup=BeautifulSoup(sent_4900,'lxml')
text=soup.get_text()
text=decontracted(text)
print(text)
print('='*50)


Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a dog would ever find in nature and if it did find rapeseed in nature and eat it, it would poison them. Today's Food industries have convinced the masses that Canola oil is a safe and even better oil than olive or virgin coconut, facts though say otherwise. Until the late 70's it was poisonous until they figured out a way to fix that. I still like it but it could be better.
Canot do sugar.  Have tried scores of SF Syrups.  NONE of them can touch the excellence of this product.<br /><br />Thick, delicious.  Perfect.  3 ingredients: Water, Maltitol, Natural Maple Flavor.  PERIOD.  No chemicals.  No garbage.<br /><br />Have numerous friends & family members hooked on this stuff.  My husband & son, who do NOT like "sugar free" prefer this over major label regular syrup.<br /><br />I use this as my SWEETENER in baki

In [21]:
# remove words with numbers python: https://stackoverflow.com/a/8082370/4084039
sent_1000=re.sub("\s*\d\s*"," ",sent_1000).strip()
print(sent_1000)

I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered boxes - both were expired! One expired back in    for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.


In [22]:
#removing spacial character: https//stackoverflow.com/a/5843547/4084039
sent_1500=re.sub('[^A-Za-z0-9]+'," ",sent_1500)
print(sent_1500)

Great ingredients although chicken should have been 1st rather than chicken broth the only thing I do not think belongs in it is Canola oil Canola or rapeseed is not someting a dog would ever find in nature and if it did find rapeseed in nature and eat it it would poison them Today s Food industries have convinced the masses that Canola oil is a safe and even better oil than olive or virgin coconut facts though say otherwise Until the late 70 s it was poisonous until they figured out a way to fix that I still like it but it could be better 


In [23]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [24]:
# combining all the above stundents
from tqdm import tqdm
preprocessed_reviews=[]
#tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance=re.sub(r"http\s+"," ",sentance)
    sentance=BeautifulSoup(sentance,'lxml').get_text()
    sentance=decontracted(sentance)
    sentance=re.sub("\s*\d\s*"," ",sentance).strip()
    sentance=re.sub('[^A-Za-z]+',' ',sentance)
    #https://gist.github.com/sebleiere/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 364171/364171 [02:57<00:00, 2047.30it/s]


In [25]:
print(preprocessed_reviews[1500])
print("="*100)
print(preprocessed_reviews[4900])
print("="*100)

great ingredients although chicken st rather chicken broth thing not think belongs canola oil canola rapeseed not someting dog would ever find nature find rapeseed nature eat would poison today food industries convinced masses canola oil safe even better oil olive virgin coconut facts though say otherwise late poisonous figured way fix still like could better
canot sugar tried scores sf syrups none touch excellence product thick delicious perfect ingredients water maltitol natural maple flavor period no chemicals no garbage numerous friends family members hooked stuff husband son not like sugar free prefer major label regular syrup use sweetener baking cheesecakes white brownies muffins pumpkin pies etc unbelievably delicious tell like


# 4. Featurization
4.1. Bag of Words¶

In [26]:
# BoW
from sklearn.feature_extraction.text import CountVectorizer

Count_Vect=CountVectorizer()
Count_Vect.fit(preprocessed_reviews)
print("some feature names",Count_Vect.get_feature_names()[:10])
print("="*100)

final_counts=Count_Vect.transform(preprocessed_reviews)
print("the type of count vectrizor",type(final_counts))
print("the number of unique words",final_counts.get_shape()[1])

some feature names ['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaaaaa', 'aaaaaaaaaaaa', 'aaaaaaaaaaaaa', 'aaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaa']
the type of count vectrizor <class 'scipy.sparse.csr.csr_matrix'>
the number of unique words 119806


In [27]:
print(final_counts)

  (0, 171)	1
  (0, 2981)	1
  (0, 3244)	1
  (0, 9989)	1
  (0, 11921)	3
  (0, 15851)	1
  (0, 19965)	1
  (0, 21165)	1
  (0, 31857)	1
  (0, 31902)	1
  (0, 52485)	1
  (0, 54033)	1
  (0, 58974)	1
  (0, 59266)	1
  (0, 60651)	1
  (0, 61495)	1
  (0, 61544)	1
  (0, 62685)	1
  (0, 64969)	1
  (0, 70189)	1
  (0, 85937)	2
  (0, 86583)	1
  (0, 89687)	1
  (0, 95025)	1
  (0, 95244)	1
  :	:
  (364170, 15058)	1
  (364170, 15090)	1
  (364170, 21165)	1
  (364170, 27803)	1
  (364170, 31267)	1
  (364170, 35735)	1
  (364170, 37652)	1
  (364170, 43361)	1
  (364170, 44689)	1
  (364170, 47659)	1
  (364170, 49722)	1
  (364170, 57226)	1
  (364170, 61549)	1
  (364170, 65803)	1
  (364170, 68347)	1
  (364170, 83675)	1
  (364170, 88911)	1
  (364170, 89591)	1
  (364170, 90394)	1
  (364170, 93051)	2
  (364170, 94215)	1
  (364170, 96513)	1
  (364170, 97379)	1
  (364170, 105335)	1
  (364170, 114833)	1


# 4.2 bi-gram and n-gram¶

In [28]:
# bi-gram , tri-gram and n-gram
#removing stop words like " not" should be avoiding before building n-grams
# Count_Vect =CountVectorizer(ngram-range=(1,2))
# you can choose these numbers min=df=10
# reading documectation : https//scikit-learn.org + sklearn.feature_extraction.text.CountVectorizer

Count_Vect=CountVectorizer(ngram_range=(1,2),min_df=10,max_features=10000)
final_bigram_Count=Count_Vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer",type(final_bigram_Count))
print("the shape of out text Bow vectorizer",final_bigram_Count.get_shape())
print("the number of unique words including both unigram and bigrams",final_bigram_Count.get_shape()[1])


the type of count vectorizer <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text Bow vectorizer (364171, 10000)
the number of unique words including both unigram and bigrams 10000


In [29]:
print(final_bigram_Count)

  (0, 4898)	1
  (0, 842)	3
  (0, 5189)	1
  (0, 8077)	2
  (0, 4638)	1
  (0, 5028)	1
  (0, 1181)	1
  (0, 2443)	1
  (0, 196)	1
  (0, 251)	1
  (0, 4662)	1
  (0, 4315)	1
  (0, 7490)	1
  (0, 5030)	1
  (0, 5705)	1
  (0, 9810)	1
  (0, 1452)	1
  (0, 9750)	1
  (0, 693)	1
  (0, 8294)	1
  (0, 1)	1
  (0, 5357)	1
  (0, 1598)	1
  (1, 8077)	1
  (1, 5030)	1
  :	:
  (364170, 5081)	1
  (364170, 6992)	1
  (364170, 4163)	1
  (364170, 2712)	1
  (364170, 7440)	1
  (364170, 8865)	1
  (364170, 5551)	1
  (364170, 9534)	1
  (364170, 503)	1
  (364170, 356)	1
  (364170, 8006)	1
  (364170, 7481)	1
  (364170, 3704)	1
  (364170, 1106)	1
  (364170, 7816)	1
  (364170, 2902)	1
  (364170, 4006)	1
  (364170, 1109)	1
  (364170, 2906)	1
  (364170, 1107)	1
  (364170, 7718)	2
  (364170, 2092)	1
  (364170, 5082)	1
  (364170, 7519)	1
  (364170, 2094)	1


# 4.3 tf-idf

In [30]:
tf_idf_Vect=TfidfVectorizer(ngram_range=(1,2),min_df=10)
tf_idf_Vect.fit(preprocessed_reviews)

print("some sample featues (unique words in the corpus) ",tf_idf_Vect.get_feature_names()[0:10])
print("="*100)

final_tf_idf=tf_idf_Vect.transform(preprocessed_reviews)
print("the type of count vectorizer",type(final_tf_idf))
print("the shape of out text Tf-idf vectorizer", final_tf_idf.get_shape())
print("the number of unique words including both unigram and biagram", final_tf_idf.get_shape()[1])

some sample featues (unique words in the corpus)  ['aa', 'aaa', 'aaaaa', 'aaah', 'aafco', 'ab', 'aback', 'abandon', 'abandoned', 'abbey']
the type of count vectorizer <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text Tf-idf vectorizer (364171, 209857)
the number of unique words including both unigram and biagram 209857


# 4.4 word2vec

In [31]:
# using Google News word2Vectors

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from gensim.test.utils import datapath
# in this project we are using a pretrained model by google
#its 3.3gb  file once you load this into your memory
# to use this code_snippet , download googleNews-Vectors-nedative300.bin
#it is 1.9GB in size
#model=KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [32]:
i=0
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

In [33]:
is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True
print("1st part")
print("="*100)

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,vector_size=5, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    print("2nd part")
    print("="*100)
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
        print("3rd part")
        print("="*100)
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")
        print("4th part")
        print("="*100)

1st part
[('shines', 0.9962291121482849), ('deeper', 0.9961068630218506), ('snappy', 0.9918104410171509), ('transforms', 0.9907973408699036), ('succulent', 0.9880290031433105), ('garlicky', 0.9875878095626831), ('hamsters', 0.9866319298744202), ('tanginess', 0.9846508502960205), ('deliciousness', 0.9846010208129883), ('mocks', 0.9838836193084717)]
[('addict', 0.9964828491210938), ('sophisticated', 0.9950692057609558), ('associate', 0.9915167689323425), ('accustomed', 0.9896323680877686), ('gummi', 0.9891809821128845), ('connoisseur', 0.9883655905723572), ('licorice', 0.9877251982688904), ('bear', 0.9835488200187683), ('acquired', 0.9829099178314209), ('personal', 0.97968590259552)]
2nd part
